# Examples for aiida-optimize 

For details on how to set up AiiDA for use with Jupyter, please refer to [the AiiDA documentation](https://aiida.readthedocs.io/projects/aiida-core/en/latest/intro/installation.html#using-aiida-in-jupyter).

In [1]:
%aiida

Loaded AiiDA DB environment.

In [2]:
import math

from aiida import orm
from aiida.engine import calcfunction, run

## Simple bisection example

First, we define a `calcfunction` that contains our objective function. Here, we use a simple exponential
$$
f(x) = c + e^{x}
$$

In [3]:
@calcfunction
def exponential(x, c):
    return c + math.exp(x.value)

In [4]:
exponential(x=Float(1), c=Float(0.))

<Float: uuid: 31d3030b-f3ad-4253-80e0-b2279e381c11 (pk: 1562) value: 2.718281828459>

---

The `OptimizationWorkChain` implements the main logic of `aiida-optimize` - it is used for all optimization tasks.

In [5]:
from aiida_optimize import OptimizationWorkChain

In [6]:
builder = OptimizationWorkChain.get_builder()

To task of defining the steps of the optimization is handled by optimization "engines" - implemented as regular Python classes. Here, we use the `Bisection` engine:

In [7]:
from aiida_optimize.engines import Bisection

The optimization engine is passed to `OptimizationWorkChain` as an input:

In [8]:
builder.engine = Bisection

We also need to specify that we want to use the `exponential` as the objective function:

In [9]:
builder.evaluate_process = exponential

We want to optimize for `x`, while keeping `c` constant. The inputs that are kept constant need to be given in the `evaluate` input namespace:

In [10]:
builder.evaluate = {'c': orm.Float(-2.)}

The optimization engine may need additional inputs. These are given in the `engine_kwargs` input. To see which options are available, you can check the documentation of the `Bisection`:

In [11]:
Bisection?

In [12]:
builder.engine_kwargs = Dict(dict=dict(lower=-1, upper=1))

---

Now we're ready to run the optimization

In [13]:
res = run(builder)

07/01/2020 07:14:59 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1567|OptimizationWorkChain|create_optimizer]: Creating optimizer instance.
07/01/2020 07:14:59 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1567|OptimizationWorkChain|not_finished]: Checking if optimization is finished.
07/01/2020 07:14:59 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1567|OptimizationWorkChain|launch_evaluations]: Launching pending evaluations.
07/01/2020 07:14:59 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1567|OptimizationWorkChain|launch_evaluations]: Launching evaluation 0
07/01/2020 07:14:59 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1567|OptimizationWorkChain|launch_evaluations]: Launching evaluation 1
07/01/2020 07:14:59 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1567|OptimizationWorkChain|get

07/01/2020 07:15:02 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1567|OptimizationWorkChain|get_results]: Retrieving output for evaluation 9
07/01/2020 07:15:02 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1567|OptimizationWorkChain|not_finished]: Checking if optimization is finished.
07/01/2020 07:15:02 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1567|OptimizationWorkChain|launch_evaluations]: Launching pending evaluations.
07/01/2020 07:15:02 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1567|OptimizationWorkChain|launch_evaluations]: Launching evaluation 10
07/01/2020 07:15:02 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1567|OptimizationWorkChain|get_results]: Checking finished evaluations.
07/01/2020 07:15:02 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1567|OptimizationWorkChain|

07/01/2020 07:15:04 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1567|OptimizationWorkChain|launch_evaluations]: Launching pending evaluations.
07/01/2020 07:15:04 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1567|OptimizationWorkChain|launch_evaluations]: Launching evaluation 19
07/01/2020 07:15:04 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1567|OptimizationWorkChain|get_results]: Checking finished evaluations.
07/01/2020 07:15:04 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1567|OptimizationWorkChain|get_results]: Retrieving output for evaluation 19
07/01/2020 07:15:04 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1567|OptimizationWorkChain|not_finished]: Checking if optimization is finished.
07/01/2020 07:15:04 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1567|OptimizationWorkChain

The optimization workchain itself has two outputs: The output of the optimal process, and its UUID.

In [14]:
res

{'optimal_process_output': <Float: uuid: 181a0344-36bf-4f31-8b55-c15627491640 (pk: 1636) value: -9.4986478504921e-07>,
 'optimal_process_uuid': <Str: uuid: 95ae3793-4990-4f03-89ab-b16e54958c0f (pk: 1637) value: 35475559-971e-437c-aa70-39563b7ac310>}

From the UUID, we can easily load the optimal process and inspect it:

In [15]:
opt_process = load_node(res['optimal_process_uuid'].value)

In [16]:
opt_process.inputs.x

<Float: uuid: 0b64cdca-e83f-4e04-9c32-253e5e5927ae (pk: 1634) value: 0.69314670562744>

## More complex example

Next, let us tackle a more complicated example: The WorkChain defined below implements the potential
$$ f(x, y, z) = x^2 + y^2 + z^2$$

To make things more interesting, each of the inputs is given in a different way:
- `x` is a simple input
- `y` is inside an input namespace
- `z` is a `Dict`, and the actual value is its `['value']`

This is more representative of optimizing a (potentially comlex) real-world process.

In [17]:
from aiida.engine import WorkChain

In [18]:
class Potential(WorkChain):
    @classmethod
    def define(cls, spec):
        super().define(spec)
        spec.input('x', valid_type=orm.Float)
        spec.input('namespace.y', valid_type=orm.Float)
        spec.input('z', valid_type=orm.Dict)
        
        spec.output('res', valid_type=orm.Float)
        
        spec.outline(
            cls.run_calculation
        )
    
    def run_calculation(self):
        # Note: the following line breaks data provenance - the calculation should
        # happen in either a calcfunction or a CalcJob. It's used here just for
        # simplicity
        self.out('res', (self.inputs.x**2 + self.inputs.namespace['y']**2 + self.inputs.z['value']**2).store())

In [19]:
run(Potential, x=orm.Float(1.), namespace={'y': orm.Float(2.)}, z=orm.Dict(dict=dict(value=3.)))

{'res': <Float: uuid: b3f168dc-035f-400e-ac2d-15c48e3aa369 (pk: 1642) value: 14.0>}

---

To transform this workchain into a format that can be handled by the `OptimizationWorkChain` and the engines, `aiida-optimize` defines wrapper workchains.

The `AddInputsWorkChain` maps a single input list onto different inputs of the target process.

In [20]:
from aiida_optimize.wrappers import AddInputsWorkChain

In [21]:
add_inputs_builder = AddInputsWorkChain.get_builder()

In [22]:
add_inputs_builder.sub_process = Potential

In [23]:
add_inputs_builder.added_input_keys = orm.List(list=['x', 'namespace.y', 'z:value'])

In [24]:
add_inputs_builder.added_input_values = orm.List(list=[1., 2., 3.])

In [25]:
run(add_inputs_builder)

07/01/2020 07:15:06 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1646|AddInputsWorkChain|run_process]: Merging inputs for the sub-process.
07/01/2020 07:15:06 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1646|AddInputsWorkChain|run_process]: Launching the sub-process.
07/01/2020 07:15:07 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1646|AddInputsWorkChain|finalize]: Retrieving outputs of the sub-process.


{'res': <Float: uuid: 75676f0b-92c6-4d2c-8234-8e69deb861d8 (pk: 1651) value: 14.0>}

---

In [26]:
builder = OptimizationWorkChain.get_builder()

The `evaluate_process` is now `AddInputsWorkChain`:

In [27]:
builder.evaluate_process = AddInputsWorkChain

The `added_input_keys` and `sub_process` passed to the `AddInputsWorkChain` remain constant.

**Important note:** Here, we need to transform the `Potential` into a `Str` using `aiida_tools.process_inputs.get_fullname`. Previously this was done automatically, but because of the added indirection this is no longer possible.

In [28]:
from aiida_tools.process_inputs import get_fullname

builder.evaluate = {
    'added_input_keys': orm.List(list=['x', 'namespace.y', 'z:value']),
    'sub_process': get_fullname(Potential)
}

Since `Bisection` can only handle one-dimensional problems, we use the `NelderMead` engine here instead.

In [30]:
from aiida_optimize.engines import NelderMead

builder.engine = NelderMead

In [31]:
NelderMead?

In [32]:
builder.engine_kwargs = orm.Dict(dict=dict(
    simplex=[[1., 0., 0.], [0., 1., 0.], [0., 0., 1.], [1., 1., 0.]],
    input_key='added_input_values',
    result_key='res',
    xtol=0.1,
    ftol=0.01
))

In [33]:
res = run(builder)

07/01/2020 07:15:42 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|create_optimizer]: Creating optimizer instance.
07/01/2020 07:15:42 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|not_finished]: Checking if optimization is finished.
07/01/2020 07:15:42 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|launch_evaluations]: Launching pending evaluations.
07/01/2020 07:15:42 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|submit_initialize]: Submitting initialization step.
07/01/2020 07:15:42 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|launch_evaluations]: Launching evaluation 0
07/01/2020 07:15:42 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkC

07/01/2020 07:15:50 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|launch_evaluations]: Launching evaluation 6
07/01/2020 07:15:50 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1701|AddInputsWorkChain|run_process]: Merging inputs for the sub-process.
07/01/2020 07:15:50 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1701|AddInputsWorkChain|run_process]: Launching the sub-process.
07/01/2020 07:15:51 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1701|AddInputsWorkChain|finalize]: Retrieving outputs of the sub-process.
07/01/2020 07:15:52 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|get_results]: Checking finished evaluations.
07/01/2020 07:15:52 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|get_results]: Retrievin

07/01/2020 07:16:01 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|check_finished]: Maximum function difference: 0.4765946502057701
07/01/2020 07:16:01 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|launch_evaluations]: Launching evaluation 11
07/01/2020 07:16:01 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1736|AddInputsWorkChain|run_process]: Merging inputs for the sub-process.
07/01/2020 07:16:01 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1736|AddInputsWorkChain|run_process]: Launching the sub-process.
07/01/2020 07:16:03 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1736|AddInputsWorkChain|finalize]: Retrieving outputs of the sub-process.
07/01/2020 07:16:03 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|ge

07/01/2020 07:16:13 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|check_finished]: Maximum distance value for the simplex: 0.6333673439535922
07/01/2020 07:16:13 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|check_finished]: Maximum function difference: 0.09370177469136001
07/01/2020 07:16:13 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|launch_evaluations]: Launching evaluation 16
07/01/2020 07:16:13 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1771|AddInputsWorkChain|run_process]: Merging inputs for the sub-process.
07/01/2020 07:16:13 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1771|AddInputsWorkChain|run_process]: Launching the sub-process.
07/01/2020 07:16:14 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] 

07/01/2020 07:16:24 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|launch_evaluations]: Launching pending evaluations.
07/01/2020 07:16:24 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|submit_inside_contraction]: Submitting inside contraction step.
07/01/2020 07:16:24 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|launch_evaluations]: Launching evaluation 21
07/01/2020 07:16:25 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1806|AddInputsWorkChain|run_process]: Merging inputs for the sub-process.
07/01/2020 07:16:25 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1806|AddInputsWorkChain|run_process]: Launching the sub-process.
07/01/2020 07:16:26 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1806|AddInputsWorkChain|f

07/01/2020 07:16:36 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|not_finished]: Checking if optimization is finished.
07/01/2020 07:16:36 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|launch_evaluations]: Launching pending evaluations.
07/01/2020 07:16:36 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|submit_inside_contraction]: Submitting inside contraction step.
07/01/2020 07:16:36 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|launch_evaluations]: Launching evaluation 26
07/01/2020 07:16:36 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1841|AddInputsWorkChain|run_process]: Merging inputs for the sub-process.
07/01/2020 07:16:36 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1841|AddIn

07/01/2020 07:16:48 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|not_finished]: Checking if optimization is finished.
07/01/2020 07:16:48 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|launch_evaluations]: Launching pending evaluations.
07/01/2020 07:16:48 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|check_finished]: Maximum distance value for the simplex: 0.1531894705187149
07/01/2020 07:16:48 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|check_finished]: Maximum function difference: 0.0107203028389704
07/01/2020 07:16:48 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1657|OptimizationWorkChain|launch_evaluations]: Launching evaluation 31
07/01/2020 07:16:48 PM <34801> aiida.orm.nodes.process.workflow.workchain.WorkC

In [34]:
load_node(res['optimal_process_uuid'].value).inputs.added_input_values

<List: uuid: 6921df5c-a6ff-4592-b419-d7912b54c389 (pk: 1896) value: [0.0013929819230568, -0.032789104224374, 0.033942234625789]>